# Web Scraping with Beautiful Soup: Solutions

In [ ]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')

## 🥊 Challenge 1: Find h1

We can also use `find()` to find the first available tag in this HTML. Use it to find the `h1` tag in the soup!


In [60]:
# YOUR CODE HERE
soup.find_all('h1')


[<h1 class="header">Welcome to the Sample Page</h1>]

## 🥊 Challenge 2: Find All

Use Beautiful Soup to find all the `a` elements with class `mainmenu`.

In [ ]:
soup.select("a.mainmenu")

## 🥊 Challenge 3: Extract Specific Attributes

Extract all `href` attributes for each `mainmenu` URL.

In [ ]:
[link['href'] for link in soup.select("a.mainmenu")]